# Exercício 1: Análise de Sentimento com Classificadores

O seguinte notebook visa demonstrar como analisar sentimentos de reviews de filmes utilizando classificadores básicos. A base de dados que usaremos será a de review de filmes do IMDb disponibilizada pela Stanford (http://ai.stanford.edu/~amaas/data/sentiment/).

Primeiramente, vamos importar os pacotes que usaremos no notebook

In [1]:
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import numpy as np

Inicialmente, temos que carregar nossa base de dados. A base de dados do IMDB é dividida em arquivos. Os arquivos que estamos utilizando já foram combinados da forma correta e separados em teste e treinamento. Para acessar os arquivos, descompactar o arquivo movie_reviews.zip para a pasta data

In [2]:
reviews_train = []
for line in open('data/full_train.txt', 'r', encoding="utf8"):
    reviews_train.append(line.strip())
    
reviews_test = []
for line in open('data/full_test.txt', 'r', encoding="utf8"):
    reviews_test.append(line.strip())

Temos que tratar a base antes de utilizar, removendo possíveis pontuações e tags html que porventura existam no texto.

In [3]:
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
    
    return reviews

reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean  = preprocess_reviews(reviews_test)

In [4]:
print(np.array(reviews_train_clean).shape)
print(np.array(reviews_test_clean).shape)

(25000,)
(25000,)


Precisamos converter as entradas em uma sacola de palavras (bag of words). Para isso usaremos a vetorização de frequência relativa (CountVectorizer). Complete o código abaixo:

In [6]:
#complete a linha abaixo
cv = CountVectorizer()
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)
X_test = cv.transform(reviews_test_clean)

Cada coluna indica uma palavra e cada linha, uma entrada de texto, assim teremos uma matriz esparsa (grande quantidade de zeros espalhados pela matriz). Note que o resultado do CountVectorizer é uma matriz esparsa.

In [7]:
X

<25000x92715 sparse matrix of type '<class 'numpy.int64'>'
	with 3461902 stored elements in Compressed Sparse Row format>

A base de dados foi organizada da seguinte forma: as primeiras 12500 entradas são de reviews positivos, o restante é de reviews negativos. Isso vale para treinamento e teste. Assim, criaremos uma lista com o valor 1 para os primeiros 12500 registros e zero para os restantes em uma faixa de 25000 (total de registros tanto em treinamento quanto em teste). 

Aproveitamos e fazemos o split da base, em dados de treinamento e validação, para isso, complete a linha abaixo:

In [8]:
target = [1 if i < 12500 else 0 for i in range(25000)]

#complete a linha abaixo
X_train, X_val, y_train, y_val = train_test_split(X, target, train_size=0.8)

In [9]:
print(X_train.shape, X_val.shape)
print(np.array(y_train).shape, np.array(y_val).shape)

(20000, 92715) (5000, 92715)
(20000,) (5000,)


Vamos treinar uma regressão logística e ver qual será a acurácia, isto é, o quão preciso é nosso modelo na pequena base de validação. Para isso, utilize o LogisticRegression e complete as linhas abaixo:

In [11]:
#complete a linha abaixo
lr = LogisticRegression(max_iter=1000)
#complete a linha abaixo para treinar o modelo
lr.fit(X_train, y_train)
print ("Accuracy : %s" % (np.round(accuracy_score(y_val, lr.predict(X_val)),4)))

Accuracy : 0.8738


Faremos o mesmo para um classificador Naive Bayes. Complete as linhas abaixo utilizando o MultinomialNB:

In [12]:
#complete a linha abaixo
nb = MultinomialNB()
#complete a linha abaixo para treinar o modelo
nb.fit(X_train, y_train)
print("Accuracy : %s" % (np.round(accuracy_score(y_val, nb.predict(X_val)),4)))

Accuracy : 0.8494


Agora vamos ver como os modelos se comportam com nossa base de teste. Primeiramente a regressão logística:

In [13]:
print ("Accuracy : %s" % (np.round(accuracy_score(target, lr.predict(X_test)),4)))

Accuracy : 0.8662


E agora, o Naive Bayes:

In [14]:
print("Accuracy : %s" % (np.round(accuracy_score(target, nb.predict(X_test)),4)))

Accuracy : 0.8142


Vamos tentar otimizar o modelo, aplicando a ideia do Naive Bayes Booleano. Para isso, precisamos transformar nossas features em dados binários. Abaixo, crie um código para binarizar nosso vetor de contagem, isto é, limitar todas as contagens para 1.
Dica: o CountVectorizer pode binarizar o vetor

In [15]:
#Complete o código abaixo
cv_binary = CountVectorizer(binary=True)
cv_binary.fit(reviews_train_clean)
X_binary      = cv_binary.transform(reviews_train_clean)
X_test_binary = cv_binary.transform(reviews_test_clean)
print(X_binary.shape, X_test_binary.shape)

(25000, 92715) (25000, 92715)


Vamos separar os conjuntos novamente em treinamento e validação

In [16]:
target = [1 if i < 12500 else 0 for i in range(25000)]

X_train, X_val, y_train, y_val = train_test_split(X_binary, target, train_size = 0.8)

Agora treine um classificador Naive Bayes para classificar nossas features binárias

In [17]:
#complete o código abaixo criando o classificador e o treinando
nb = MultinomialNB()
nb.fit(X_train, y_train)

MultinomialNB()

In [18]:
print("Accuracy : %s" % (accuracy_score(y_val, nb.predict(X_val))))

Accuracy : 0.8544


In [19]:
print(np.array(y_val).shape, X_val.shape)
print(np.array(target).shape, X_test_binary.shape)

(5000,) (5000, 92715)
(25000,) (25000, 92715)


Vamos agora ver a acurácia no nosso conjunto de teste

In [20]:
print("Accuracy : %s" % (accuracy_score(target, nb.predict(X_test_binary))))

Accuracy : 0.82396
